In [ ]:
import copy
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
from mpl_toolkits.basemap import Basemap
import seaborn as sns


https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
# read stored data from data_preparation notebook
# @formatter:off
%store -r df_airports
%store -r airports_dict
%store -r df_merged
# @formatter:on
GG = nx.read_gml('Graphs/airlines.gml')


    centralities
    degree distribution

In [ ]:
degree_centrality = nx.degree_centrality(GG)
closeness_centrality = nx.closeness_centrality(GG)
betweenness_centrality = nx.betweenness_centrality(GG)
eigenvector_centrality = nx.eigenvector_centrality(GG)


In [ ]:
nx.set_node_attributes(GG, closeness_centrality, 'closeness centrality')
nx.set_node_attributes(GG, degree_centrality, 'degree centrality')
nx.set_node_attributes(GG, betweenness_centrality, 'betweenness centrality')
nx.set_node_attributes(GG, eigenvector_centrality, 'eigenvector centrality')


In [ ]:
centralityArr = [degree_centrality, closeness_centrality,
                 betweenness_centrality, eigenvector_centrality]
centralityNames = ['degree', 'closeness', 'betweenness', 'eigenvector']


In [ ]:
map = Basemap(projection='cyl', resolution=None,
              llcrnrlat=-90, urcrnrlat=90,
              llcrnrlon=-180, urcrnrlon=180, )

lats = [k[1]['Latitude'] for k in airports_dict.items()]
longs = [k[1]['Longitude'] for k in airports_dict.items()]

# convert long/lat into map proj https://matplotlib.org/basemap/users/mapcoords.html
pos = {}
for n in GG.nodes():
    x,y = map(GG.nodes[n]['Longitude'], GG.nodes[n]['Latitude'])
    pos[n] = (x,y)

## Centralities

In [ ]:
for centrality, name in zip(centralityArr, centralityNames):
    print(name)
    minCentralityVal = min(centrality.values())
    maxCentralityVal = max(centrality.values())
    print(minCentralityVal, maxCentralityVal)
    # normalize values into [0,1] for color map
    ff = np.array(list(nx.get_node_attributes(
        GG, f'{name} centrality').values()))
    CC = list((ff - minCentralityVal) / (maxCentralityVal - minCentralityVal))

    fig = plt.figure(dpi=1000, figsize=(8, 6))
    map = Basemap(projection='cyl', resolution=None,
                  llcrnrlat=-90, urcrnrlat=90,
                  llcrnrlon=-180, urcrnrlon=180, )

    map.bluemarble(alpha=0.9)
    # draw twice: first one is lines only with alpha 0.1, second one is nodes only
    nx.draw_networkx(GG, pos=pos, node_size=1, width=0.005, node_color=CC, vmin=min(CC), vmax=max(CC),
                     cmap=plt.cm.get_cmap('viridis'), arrows=False, with_labels=False, edge_color='w', alpha=0.1,
                     node_shape='.')
    nx.draw_networkx(GG, pos=pos, node_size=1, width=0.0, node_color=CC, vmin=min(CC), vmax=max(CC),
                     cmap=plt.cm.get_cmap('viridis'), arrows=False, with_labels=False, edge_color='w', alpha=0.8,
                     node_shape='.')

    plt.title(f'{name} Centrality')
    plt.savefig(f'Figures/{name.capitalize()}_centrality_world.pdf', dpi=1000)


## Degree Distribution

In [ ]:
G_DEGS = [GG.degree()[i] for i in GG.nodes()]
bins = np.logspace(np.log10(min(G_DEGS)), np.log10(max(G_DEGS) + 1), 30)
# take the average value of all degrees
AVG_VAL = np.average(G_DEGS)

In [ ]:
fig = plt.figure(dpi=300, figsize=(6, 6))
plt.hist(G_DEGS, bins=bins, density=True, alpha=0.8)
plt.axvline(x=AVG_VAL, color='r', label='Average Degree', c='orange')
plt.xlabel('Degree $k$', fontsize=14)
plt.ylabel('$\mathcal{P}(k)$', fontsize=14)
plt.title('Degree Distribution', fontsize=20)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.savefig('Figures/degree_distribution.pdf', dpi=1000)
plt.show()